In [26]:
import pandas as pd
import numpy as np

import pickle

from bs4 import BeautifulSoup
import requests
import time
import re
import string

import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys

import os

In [2]:
# timing function
def timefunc(f):
    def f_timer(*args, **kwargs):
        start = time.time()
        result = f(*args, **kwargs)
        end = time.time()
        print(f.__name__, 'took', end - start, 'seconds')
        return result
    return f_timer

### Get URL list

In [3]:
url = 'https://www.youtube.com/playlist?list=UUAuUUnT6oDeKwE6v1NGQxug'

In [4]:
chromedriver = "/Users/THoshino/anaconda/bin/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver
driver = webdriver.Chrome(chromedriver)
driver.get(url)
time.sleep(2)
driver.execute_script("window.scrollTo(0,40000000000);")

In [5]:
for i in range(25):
    try:
        button = driver.find_element_by_xpath('//*[@id="pl-video-list"]/button/span/span[2]')
        button.click()
        time.sleep(2)
#         links = BeautifulSoup(driver.page_source,'lxml').find_all("div",class_="comment__body hyphens")
#         print(i)
    except:
        break

In [6]:
links = []
titles = BeautifulSoup(driver.page_source,'lxml').find_all(class_="pl-video-title")
for i in range(0,len(titles)):
    links.append(titles[i].find('a').get('href'))
links

['/watch?v=D-_Az5nZBBM&index=1&list=UUAuUUnT6oDeKwE6v1NGQxug',
 '/watch?v=GqGksNRYu8s&index=2&list=UUAuUUnT6oDeKwE6v1NGQxug',
 '/watch?v=_mq-HqRnngc&index=3&list=UUAuUUnT6oDeKwE6v1NGQxug',
 '/watch?v=urntcMUJR9M&index=4&list=UUAuUUnT6oDeKwE6v1NGQxug',
 '/watch?v=Qs9m8obl0AY&index=5&list=UUAuUUnT6oDeKwE6v1NGQxug',
 '/watch?v=Nh-TVcNFtVI&index=6&list=UUAuUUnT6oDeKwE6v1NGQxug',
 '/watch?v=2ZviHInGBJQ&index=7&list=UUAuUUnT6oDeKwE6v1NGQxug',
 '/watch?v=VQRjouwKDlU&index=8&list=UUAuUUnT6oDeKwE6v1NGQxug',
 '/watch?v=cmpu58yv8-g&index=9&list=UUAuUUnT6oDeKwE6v1NGQxug',
 '/watch?v=1S9RVl1d-l4&index=10&list=UUAuUUnT6oDeKwE6v1NGQxug',
 '/watch?v=0d6iSvF1UmA&index=11&list=UUAuUUnT6oDeKwE6v1NGQxug',
 '/watch?v=1BXYSGepx7Q&index=12&list=UUAuUUnT6oDeKwE6v1NGQxug',
 '/watch?v=WvvuLDX7iIk&index=13&list=UUAuUUnT6oDeKwE6v1NGQxug',
 '/watch?v=8nt3edWLgIg&index=14&list=UUAuUUnT6oDeKwE6v1NGQxug',
 '/watch?v=lAzQWtkPzbI&index=15&list=UUAuUUnT6oDeKwE6v1NGQxug',
 '/watch?v=iEy-xTbcr2A&index=16&list=UUAuUUnT6oDe

In [11]:
with open('youtube_ted_links.pkl','wb') as picklefile:
    pickle.dump(links,picklefile)

### Pickle pickup point

In [3]:
with open('youtube_ted_links.pkl','rb') as picklefile:
    urls = pickle.load(picklefile)

In [4]:
base_url = 'https://www.youtube.com'

In [5]:
url_list = [base_url + urls[i] for i in range(0,len(urls))]

In [16]:
@timefunc
def get_comments(url):
    try:
        chromedriver = "/Users/THoshino/anaconda/bin/chromedriver"
        os.environ["webdriver.chrome.driver"] = chromedriver
        driver = webdriver.Chrome(chromedriver)
        driver.get(url)
        time.sleep(5)
        driver.execute_script("window.scrollTo(0,400000000000);")
        time.sleep(1)

        countload = 0
        while True:
            try:
                button = driver.find_element_by_xpath('//*[@id="comment-section-renderer"]/button')
                button.click()
                time.sleep(2.5)
                countload += 1
                print("Loadmore: ", countload)
                if countload > 250:
                    break
            except:
                if "ElementNotVisibleException":
                    break
    
    except:
        return None
        driver.close()
        print("ERROR - driver closed")
    
    else:
        try:
            title_soup = BeautifulSoup(driver.page_source,'lxml').find(id="eow-title").get_text()
            title = title_soup.replace('\n','').strip()

            comments = []
            coms = BeautifulSoup(driver.page_source,'lxml').find_all('section',class_="comment-thread-renderer")

            for i in range(0,len(coms)):
                comms = coms[i].find_all('div', class_="comment-renderer-text-content")
                for j in range(0,len(comms)):
                    comment = comms[j].get_text()
                    comment = " ".join(comment.split())
                    comment = comment.replace('\n',"")

                    content = {"url": url,
                               "title": title,
                               "comment": comment}

                    comments.append(content)
            
            driver.close()
            print("SUCCESS - driver closed")
            return comments
        
        except:
            print("ERROR - no content")
            driver.close()
            pass

In [7]:
url = url_list[1]
get_comments(url)

Loadmore:  1
Loadmore:  2
Loadmore:  3
SUCCESS - driver closed
get_comments took 18.62955594062805 seconds


[{'comment': "Institutions are strangers. What's happened is, institutions have been exposed as liars and crooks by strangers on the Internet.\ufeff",
  'title': "We've stopped trusting institutions and started trusting strangers | Rachel Botsman",
  'url': 'https://www.youtube.com/watch?v=GqGksNRYu8s&index=2&list=UUAuUUnT6oDeKwE6v1NGQxug'},
 {'comment': '+happyflea First solution I\'d like to propose is pretty much a direct slap at what seems to be the source of the lack of truth-telling and effectivization on part of both the state and other institutions, and it comes in a top-level workover of the separation of powers and such. Basically, firstly you need to rewrite the constitution to say that "The country must be run with a mind to sustainability, and for the sake of the furtherance of scientific understanding and orderly progress, so as to insure the peace and prosperity of all peoples; any law, organization or individual action determined beyond doubt to act contrary to this sha

In [17]:
url_list_500 = url_list[:500]
url_list_1000 = url_list[500:1000]
url_list_1500 = url_list[1000:1500]
url_list_2000 = url_list[1500:2000]
url_list_2500 = url_list[2000:]

In [18]:
@timefunc
def iterate_urls_500(url_list_500):
    c_list = []
    for count, url in enumerate(url_list_500):
        time.sleep(0.05)
        comments = get_comments(url)
        if comments: 
            c_list.append(comments)
            with open("ytcomments500_v2.pkl", "wb") as picklefile:
                pickle.dump(c_list, picklefile)
            print(count)
        else:
            print('No comments')
            continue
        if count % 50 == 0:
            time.sleep(1)
            print('Processing Next 50')
    
    return c_list

In [19]:
@timefunc
def iterate_urls_1000(url_list_1000):
    c_list = []
    for count, url in enumerate(url_list_1000):
        time.sleep(0.05)
        comments = get_comments(url)
        if comments: 
            c_list.append(comments)
            with open("ytcomments1000_v2.pkl", "wb") as picklefile:
                pickle.dump(c_list, picklefile)
            print(count)
        else:
            print('No comments')
            continue
        if count % 50 == 0:
            time.sleep(1)
            print('Processing Next 50')
    
    return c_list

In [20]:
@timefunc
def iterate_urls_1500(url_list_1500):
    c_list = []
    for count, url in enumerate(url_list_1500):
        time.sleep(0.05)
        comments = get_comments(url)
        if comments: 
            c_list.append(comments)
            with open("ytcomments1500_v2.pkl", "wb") as picklefile:
                pickle.dump(c_list, picklefile)
            print(count)
        else:
            print('No comments')
            continue
        if count % 50 == 0:
            time.sleep(1)
            print('Processing Next 50')
    
    return c_list

In [21]:
@timefunc
def iterate_urls_2000(url_list_2000):
    c_list = []
    for count, url in enumerate(url_list_2000):
        time.sleep(0.05)
        comments = get_comments(url)
        if comments:
            c_list.append(comments)
            with open("ytcomments2000_v2.pkl", "wb") as picklefile:
                pickle.dump(c_list, picklefile)
            print(count)
        else:
            print('No comments')
            continue
        if count % 50 == 0:
            time.sleep(1)
            print('Processing Next 50')
    
    return c_list

In [22]:
@timefunc
def iterate_urls_2500(url_list_2500):
    c_list = []
    for count, url in enumerate(url_list_2500):
        time.sleep(0.05)
        comments = get_comments(url)
        if comments: 
            c_list.append(comments)
            with open("ytcomments2500_v2.pkl", "wb") as picklefile:
                pickle.dump(c_list, picklefile)
            print(count)
        else:
            print('No comments')
            continue
        if count % 50 == 0:
            time.sleep(1)
            print('Processing Next 50')
    
    return c_list

In [23]:
# it_500 = iterate_urls_500(url_list_500)

In [ ]:
it_1000 = iterate_urls_1000(url_list_1000)

Loadmore:  1
Loadmore:  2
Loadmore:  3
SUCCESS - driver closed
get_comments took 19.15500497817993 seconds
0
Processing Next 50


In [ ]:
# with open("ytcomments500_v2.pkl", "rb") as picklefile:
#     df = pickle.load(picklefile)

In [ ]:
# it_1500 = iterate_urls_1500(url_list_1500)

In [ ]:
# it_2000 = iterate_urls_2000(url_list_2000)

In [ ]:
# it_2500 = iterate_urls_2500(url_list_2500)

In [236]:
# # 'long'
# url = 'https://www.youtube.com/watch?v=iG9CE55wbtY&list=UUAuUUnT6oDeKwE6v1NGQxug&index=2247'
# # 'short'
# # url = 'https://www.youtube.com/watch?v=GqGksNRYu8s&index=1&list=UUAuUUnT6oDeKwE6v1NGQxug'

In [237]:
# chromedriver = "/Users/THoshino/anaconda/bin/chromedriver"
# os.environ["webdriver.chrome.driver"] = chromedriver
# driver = webdriver.Chrome(chromedriver)
# driver.get(url)
# time.sleep(3)
# driver.execute_script("window.scrollTo(0,40000000000);")

In [238]:
# countload = 0
# while True:
#     try:
#         button = driver.find_element_by_xpath('//*[@id="comment-section-renderer"]/button')
#         button.click()
#         time.sleep(2)
#         countload += 1
#         print("Loadmore: ", countload)
#         if countload > 200:
#             break
#     except:
#         if "ElementNotVisibleException":
#             break

In [239]:
# countrep = 0
# while True:
#     try:
#         class_ = 'load-more-text'
#         button = driver.find_element_by_class_name(class_)
#         button.click()
#         countrep += 1
#         print("Loadrep: ", countrep)
#     except:
#         if "ElementNotVisibleException":
#             break
# # <div class="yt-uix-expander-head comment-replies-renderer-expander-down comment-replies-renderer-view" tabindex="0">
# #         View all 14 replies
# #       </div>

In [240]:
# comments = []
# coms = BeautifulSoup(driver.page_source,'lxml').find_all('section',class_="comment-thread-renderer")
# for i in range(0,len(coms)):
#     comms = coms[i].find_all('div', class_="comment-renderer-text-content")
#     for j in range(0,len(comms)):
#         comment = comms[j].get_text()
#         comment = " ".join(comment.split())
#         comment = comment.replace('\n',"")
#         comments.append(comment)
# len(comments)

In [241]:
# title_soup = BeautifulSoup(driver.page_source,'lxml').find(id="eow-title").get_text()
# title = title_soup.replace('\n','').strip()